In [1]:
from astropy.io import fits
from astropy.table import Table

In [21]:
hdus = fits.open("../tests/data/EUVEngc4151imgx.fits")

In [23]:
hdus[3].header

XTENSION= 'BINTABLE'           / Binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =                   20 / Width of table in bytes                        
NAXIS2  =                    2 / The number of rows                             
PCOUNT  =                    0 / Size of special data area                      
GCOUNT  =                    1 / Only one group                                 
TFIELDS =                    3 / Number of fields per row                       
EXTNAME = 'mw_limits'          / Name of table                                  
                                                                                
TTYPE1  = 'NAME    '           / Name of limited value                          
TFORM1  = '12A     '           /                                                
TUNIT1  = '        '        

In [18]:
Table.read("../tests/data/outputs/EUVEngc4151imgx_written_by_astrors.fits", hdu=2)

NAME,LOW
bytes13,float32
swadct,3.0
B lookzen,1.3563156e-19


In [17]:
Table.read("../tests/data/EUVEngc4151imgx.fits", hdu=2)

NAME,LOW,HIGH
,UNKNOWN,UNKNOWN
bytes12,float32,float32
swadct,3,80
lookzen,0,102


In [4]:
bytes = open("../tests/data/EUVEngc4151imgx.fits", "rb").read()

In [5]:
def get_first_header_size(bytes):
    # Header may be multiple of 2880 bytes, get the size of the header until the block where the END card is present
    header_size = 0
    while True:
        header_size += 2880
        if b"END     " in bytes[header_size:header_size + 2880]:
            header_size += 2880
            break
        
    return header_size


In [6]:
def get_data_size_in_bytes(header):
    ## It may have multiple NAXISn keywords, so we need to get the product of all of them
    size_in_bytes = 1
    for i in range(1, header["NAXIS"] + 1):
        size_in_bytes *= header["NAXIS" + str(i)]
    
    size_in_bytes *= abs(header["BITPIX"] / 8)
    ## Make it multiple of 2880 bytes
    if size_in_bytes % 2880 != 0:
        size_in_bytes += 2880 - (size_in_bytes % 2880)
    
    return int(size_in_bytes)
    

In [25]:
second_hdus_start = get_first_header_size(bytes) + get_data_size_in_bytes(hdus[0].header)
second_hdu_bytes = bytes[second_hdus_start:]

In [27]:
second_hdu_bytes

b'RA_PROC =    182.6379100000007 / R.A. used to process data (degrees)            DEC_PROC=             39.41343 / Declination used to process data (degrees)     OBSERVER= \'A. A. Zdziarski\'    / Original observing P.I. (EUVE = calibration)   DATE-OBS= \'30/04/97 GMT\'       / Start date of observation (dd/mm/yy) 19yy      TIME-OBS= \'23:51:30 GMT\'       / Start time of observation (hh:mm:ss GMT)       DATE-END= \'07/05/97 GMT\'       / End date of observation (dd/mm/yy) 19yy        TIME-END= \'09:34:27 GMT\'       / End time of observation (hh:mm:ss GMT)         OBS_MODE= \'POINTING\'           / Inertial pointing mode                         DITHER  = \'NONE    \'           / Spacecraft dither type (DITHERED, SPIRAL, NONE)DETMODE = \'WSZ     \'           / Detector position conversion mode (WSZ or XY)  OFF-AXIS=                    T / Was this pointing done off-axis                MOVING  =                    F / Did the source position vary during observationDAYNIGHT= \'NIGHT   \'

In [20]:
Table.read("../tests/data/EUVEngc4151imgx.fits", hdu=1)

NAME,LOW,HIGH
,UNKNOWN,UNKNOWN
bytes8,float32,float32
dsq1sf,0,50
dsadct,3,5000
lookzen,0,102


In [44]:
data_second_hdu = bytes[2880 * 4: ]

In [21]:
class Column:
    number: int
    ttype: str
    tform: bytes
    
    def __init__(self, number, ttype, tform, tbcol):
        self.number = number
        self.ttype = ttype
        self.tform = tform
        self.tbcol = tbcol
        self.values = []
    
    def print(self):
        print(f"Column {self.number}: {self.ttype} {self.tform} {self.values}")

In [22]:
header = hdus[1].header
columns = []
for i in range(1, header["TFIELDS"] + 1):
    columns.append(Column(i, header["TTYPE" + str(i)], header["TFORM" + str(i)], header["TBCOL" + str(i)]))

In [23]:
## if tform is E15.2 I want to get the 15
def get_tform_type_size(tform):
    if len(tform.strip()) == 1:
        size = None
    else:
        size = int(tform[1:].split(".")[0])
        
    return tform[:1], size

In [24]:
pos = 0

for line in range(1, header["NAXIS2"]):
    for key, column in enumerate(columns):
        typ, size = get_tform_type_size(column.tform)
        
        if typ == "D":
            column.values.append(float(data_second_hdu[pos:pos + size]))
        elif typ == "E":
            column.values.append(float(data_second_hdu[pos:pos + size]))
        elif typ == "I":
            column.values.append(int(data_second_hdu[pos:pos + size]))
        elif typ == "A":
            column.values.append(data_second_hdu[pos:pos + size].decode("ascii"))
        else:
            column.values.append(data_second_hdu[pos:pos + size])
        
        pos += size + 1

In [25]:
columns[1].print()

Column 2: CRVAL2 D25.17 [39.39633673411, 39.41214313815, 39.410944281389995]


In [26]:
from astropy.table import Table
t = Table.read("../tests/data/WFPC2u57.fits", format="fits")
t

CRVAL1,CRVAL2,CRPIX1,CRPIX2,CD1_1,CD1_2,CD2_1,CD2_2,DATAMIN,DATAMAX,MIR_REVR,ORIENTAT,FILLCNT,ERRCNT,FPKTTIME,LPKTTIME,CTYPE1,CTYPE2,DETECTOR,DEZERO,BIASEVEN,BIASODD,GOODMIN,GOODMAX,DATAMEAN,GPIXELS,SOFTERRS,CALIBDEF,STATICD,ATODSAT,DATALOST,BADPIXEL,OVERLAP,PHOTMODE,PHOTFLAM,PHOTZPT,PHOTPLAM,PHOTBW,MEDIAN,MEDSHADO,HISTWIDE,SKEWNESS,MEANC10,MEANC25,MEANC50,MEANC100,MEANC200,MEANC300,BACKGRND
,,,,,,,,,,LOGICAL-,,,,,,CHARACTER*8,CHARACTER*8,,,,,,,,,,,,,,,,CHARACTER*48,,,,,,,,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,int64,int64,float64,float64,bytes8,bytes8,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,bytes48,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
182.6311886308,39.39633673411,420,424.5,-1.06704e-06,-1.25958e-05,-1.26016e-05,1.06655e-06,-73.19537,3777.701,T,-85.16,0,0,51229.79857399999,51229.79874200001,RA---TAN,DEC--TAN,1,316.6452,316.6715,316.6189,-5.064006,2552.17,0.4182382,632387,0,1466,0,16,0,0,0,"WFPC2,1,A2D7,LRF#4877.0,,CAL",3.44746e-16,-21.1,4884.258,20.20996,-0.175651,-0.121681,1.033711,-1.983727,0.12958,0.3129676,0.4577668,0.3916293,0.3115222,0.3295493,-0.3676353
182.6255233634,39.41214313815,423.5,414,-2.75605e-05,2.08221e-06,2.08021e-06,2.75871e-05,-19.95751,3740.623,T,4.31638,0,0,51229.79874200001,51229.79891000002,RA---TAN,DEC--TAN,2,354.0573,354.0761,354.0386,-3.379329,3720.573,0.6385469,631340,0,2072,0,25,0,0,0,"WFPC2,2,A2D7,LRF#4877.0,,CAL",3.382652e-16,-21.1,4884.261,20.20992,-0.1896807,-0.1899146,1.053666,-1.947547,0.2564496,0.1057321,0.9952926,0.8409855,0.6534808,0.5836062,0.2101429
182.6523792305,39.41094428138999,436,424.5,2.23913e-06,2.75426e-05,2.75684e-05,-2.23704e-06,-58.53249,3282.136,T,94.6434,0,0,51229.79891000002,51229.799078,RA---TAN,DEC--TAN,3,306.6676,306.6541,306.681,-3.713645,3282.136,0.5217843,633259,0,1551,0,7,0,0,0,"WFPC2,3,A2D7,LRF#4877.0,,CAL",3.372288e-16,-21.1,4884.255,20.21006,0.01109185,0.06184989,1.003236,28.21703,1.669304,0.3472891,0.327215,0.6145074,0.7027547,0.5681439,0.4761558
182.650022355,39.38946044168,423,421,2.75358e-05,-2.50931e-06,-2.50689e-06,-2.75624e-05,-72.41166,3420.24,T,-174.798,0,0,51229.799079,51229.79924599999,RA---TAN,DEC--TAN,4,311.7072,311.7648,311.6496,-3.289304,3420.24,0.7084697,628289,0,1816,0,57,0,0,0,"WFPC2,4,A2D7,LRF#4877.0,,CAL",3.46466e-16,-21.1,4884.256,20.20999,-0.06813275,-0.03253493,0.9846212,-4.591252,0.1359666,0.04015671,0.1129934,1.897491,0.9687142,1.05377,0.3466465


In [27]:
data_second_hdu[800: ]

b' 1.8262552336340000E2      3.9412143138150002E1      4.235000E2      4.140000E2    -2.756050E-5     2.082210E-6     2.080210E-6     2.758710E-5     -1.995751E1      3.740623E3 T      4.316380E0            0            0      5.1229798742000003E4      5.1229798910000017E4 RA---TAN DEC--TAN            2      3.540573E2      3.540761E2      3.540386E2     -3.379329E0      3.720573E3     6.385469E-1       631340            0         2072            0           25            0            0            0 WFPC2,2,A2D7,LRF#4877.0,,CAL                        3.382652E-16     -2.110000E1      4.884261E3      2.020992E1    -1.896807E-1    -1.899146E-1      1.053666E0     -1.947547E0     2.564496E-1     1.057321E-1     9.952926E-1     8.409855E-1     6.534808E-1     5.836062E-1     2.101429E-1      1.8265237923050001E2      3.9410944281389994E1      4.360000E2      4.245000E2     2.239130E-6     2.754260E-5     2.756840E-5    -2.237040E-6     -5.853249E1      3.282136E3 T      9.464340E1         

In [28]:
hdus[1].header

XTENSION= 'TABLE   '           / Ascii table extension                          
BITPIX  =                    8 / 8-bits per 'pixels'                            
NAXIS   =                    2 / Simple 2-D matrix                              
NAXIS1  =                  796 / Number of characters per row                   
NAXIS2  =                    4 / The number of rows                             
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
TFIELDS =                   49 / Number of fields per row                       
EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               
                                                                                
TTYPE1  = 'CRVAL1  '           /                                                
TBCOL1  =                    1 /                                                
TFORM1  = 'D25.17  '        

In [53]:
t = Table.read("../tests/data/outputs/test_table.fits")

In [54]:
t

CRVAL1,CRVAL2,CRPIX1,CRPIX2,CD1_1,CD1_2,CD2_1,CD2_2,DATAMIN,DATAMAX,MIR_REVR,ORIENTAT,FILLCNT,ERRCNT,FPKTTIME,LPKTTIME,CTYPE1,CTYPE2,DETECTOR,DEZERO,BIASEVEN,BIASODD,GOODMIN,GOODMAX,DATAMEAN,GPIXELS,SOFTERRS,CALIBDEF,STATICD,ATODSAT,DATALOST,BADPIXEL,OVERLAP,PHOTMODE,PHOTFLAM,PHOTZPT,PHOTPLAM,PHOTBW,MEDIAN,MEDSHADO,HISTWIDE,SKEWNESS,MEANC10,MEANC25,MEANC50,MEANC100,MEANC200,MEANC300,BACKGRND
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes48,float64,int64,int64,float64,float64,bytes48,bytes48,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,bytes48,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
182.6311886308,39.39633673411,420.0,424.5,-1.06704e-06,-1.25958e-05,-1.26016e-05,1.06655e-06,-73.19537,3777.701,T,-85.16,0,0,51229.798573999986,51229.798742000006,RA---TAN,DEC--TAN,1,316.6452,316.6715,316.6189,-5.064006,2552.17,0.4182382,632387,0,1466,0,16,0,0,0,"WFPC2,1,A2D7,LRF#4877.0,,CAL",3.44746e-16,-21.1,4884.258,20.20996,-0.175651,-0.121681,1.033711,-1.983727,0.12958,0.3129676,0.4577668,0.3916293,0.3115222,0.3295493,-0.3676353
182.6255233634,39.41214313815,423.5,414.0,-2.75605e-05,2.08221e-06,2.08021e-06,2.75871e-05,-19.95751,3740.623,T,4.31638,0,0,51229.798742000006,51229.79891000002,RA---TAN,DEC--TAN,2,354.0573,354.0761,354.0386,-3.379329,3720.573,0.6385469,631340,0,2072,0,25,0,0,0,"WFPC2,2,A2D7,LRF#4877.0,,CAL",3.382652e-16,-21.1,4884.261,20.20992,-0.1896807,-0.1899146,1.053666,-1.947547,0.2564496,0.1057321,0.9952926,0.8409855,0.6534808,0.5836062,0.2101429
182.6523792305,39.410944281389995,436.0,424.5,2.23913e-06,2.75426e-05,2.75684e-05,-2.23704e-06,-58.53249,3282.136,T,94.6434,0,0,51229.79891000002,51229.799078,RA---TAN,DEC--TAN,3,306.6676,306.6541,306.681,-3.713645,3282.136,0.5217843,633259,0,1551,0,7,0,0,0,"WFPC2,3,A2D7,LRF#4877.0,,CAL",3.372288e-16,-21.1,4884.255,20.21006,0.01109185,0.06184989,1.003236,28.21703,1.669304,0.3472891,0.327215,0.6145074,0.7027547,0.5681439,0.4761558
182.650022355,39.38946044168,423.0,421.0,2.75358e-05,-2.50931e-06,-2.50689e-06,-2.75624e-05,-72.41166,3420.24,T,-174.798,0,0,51229.799079,51229.79924599999,RA---TAN,DEC--TAN,4,311.7072,311.7648,311.6496,-3.289304,3420.24,0.7084697,628289,0,1816,0,57,0,0,0,"WFPC2,4,A2D7,LRF#4877.0,,CAL",3.46466e-16,-21.1,4884.256,20.20999,-0.06813275,-0.03253493,0.9846212,-4.591252,0.1359666,0.04015671,0.1129934,1.897491,0.9687142,1.05377,0.3466465


In [36]:
data_second_hdu[796: ]

b'     1.8262552336340000E2      3.9412143138150002E1      4.235000E2      4.140000E2    -2.756050E-5     2.082210E-6     2.080210E-6     2.758710E-5     -1.995751E1      3.740623E3 T      4.316380E0            0            0      5.1229798742000003E4      5.1229798910000017E4 RA---TAN DEC--TAN            2      3.540573E2      3.540761E2      3.540386E2     -3.379329E0      3.720573E3     6.385469E-1       631340            0         2072            0           25            0            0            0 WFPC2,2,A2D7,LRF#4877.0,,CAL                        3.382652E-16     -2.110000E1      4.884261E3      2.020992E1    -1.896807E-1    -1.899146E-1      1.053666E0     -1.947547E0     2.564496E-1     1.057321E-1     9.952926E-1     8.409855E-1     6.534808E-1     5.836062E-1     2.101429E-1      1.8265237923050001E2      3.9410944281389994E1      4.360000E2      4.245000E2     2.239130E-6     2.754260E-5     2.756840E-5    -2.237040E-6     -5.853249E1      3.282136E3 T      9.464340E1     

In [30]:
len("""SIMPLE  =                    T / FITS STANDARD                                  BITPIX  =                    8 / Character information                          NAXIS   =                    0 / No image data array present                    EXTEND  =                    T / There may be standard extensions               DATE    = '31/10/97'           / Date file was written (dd/mm/yy) 19yy          ORIGIN  = 'CEA/SSL UC Berkeley' / EUVE Science Archive                          CREATOR = 'STWFITS '           / Fitsio version 11-May-1995                     TELESCOP= 'EUVE    '           / Extreme Ultraviolet Explorer                   INSTTYPE= 'DS/S    '           / Instrument type (DS/S, SCANNER)                OBJECT  = 'NGC 4151'           / Name of observed object                        RA_OBJ  =     182.635454000001 / R.A. of the object (degrees)                   DEC_OBJ =     39.4057280000001 / Declination of the object (degrees)            RA_PNT  =     182.988000000001 / R.A. of the pointing direction (degrees)       DEC_PNT =              39.5477 / Declination of the pointing direction (degrees)RA_PROC =     182.637910000001 / R.A. used to process data (degrees)            DEC_PROC=             39.41343 / Declination used to process data (degrees)     OBSERVER= 'A. A. Zdziarski'    / Original observing P.I. (EUVE = calibration)   DATE-OBS= '30/04/97 GMT'       / Start date of observation (dd/mm/yy) 19yy      TIME-OBS= '23:51:30 GMT'       / Start time of observation (hh:mm:ss GMT)       DATE-END= '07/05/97 GMT'       / End date of observation (dd/mm/yy) 19yy        TIME-END= '09:34:27 GMT'       / End time of observation (hh:mm:ss GMT)         OBS_MODE= 'POINTING'           / Inertial pointing mode                         DITHER  = 'NONE    '           / Spacecraft dither type (DITHERED, SPIRAL, NONE)DETMODE = 'WSZ     '           / Detector position conversion mode (WSZ or XY)  OFF-AXIS=                    T / Was this pointing done off-axis                MOVING  =                    F / Did the source position vary during observationDAYNIGHT= 'NIGHT   '           / Day/night data indicator (DAY, NIGHT, BOTH)    VALIDTIM=      201378.81295777 / Amount of telemetry present (seconds)          RA_UNIT = 'deg     '           / Units for Right Ascension                      DEC_UNIT= 'deg     '           / Units for Declination                          EQUINOX =                2000. / Coordinate equinox                             RADECSYS= 'FK5     '           / Frame of reference of coordinates              TIMESYS = 'MJD     '           / MJD = JD - 2400000.5                           TIMEZERO=                   0. / No time offset required for EUVE event times   TIMEUNIT= 's       '           / Units for TSTART, TSTOP, TIMEZERO              CLOCKCOR= 'NO      '           / Not corrected to UT                            TIMEREF = 'LOCAL   '           / No corrections applied (barycentric, etc.)     TASSIGN = 'SATELLITE'          / Event times are assigned at the satellite      TSTART  =     913161090.048001 / Time of start of observation (seconds)         TSTOP   =     913714467.840001 / Time of end of observation (seconds)           MJDREF  =               40000. / MJD of SC clock start, 24.00 May 1968          EGOCSVER= 'egocs1.7.1'         / Software version used to produce this data     REFVERS = 'egodata1.15.1'      / Reference calibration dataset version used     COMMENT     ' '                                                                 COMMENT     'This file is part of the EUVE Science Archive. It contains'        COMMENT     'images and filter limits for one EUVE observation.'                COMMENT     ' '                                                                 COMMENT     'The EUVE Science Archive contains the science data from'           COMMENT     'observations performed with the EUVE telescopes. It forms one'     COMMENT     'part of the EUVE Permanent Archive. The other part of the'         COMMENT     'permanent archive is the EUVE Telemetry Archive, which is a'       COMMENT     'complete record of the raw telemetry from the EUVE mission.'       COMMENT     ' '                                                                 COMMENT     'For documentation of the contents of the EUVE Science Archive,'    COMMENT     'see the "EUVE Science Archive User's Guide".  The contents of'     COMMENT     'the EUVE Telemetry Archive are described in the "EUVE'             COMMENT     'Telemetry Archive User's Guide".'                                  COMMENT     ' '                                                                 COMMENT     'The EUVE Permanent Archive was produced by the Center for EUV'     COMMENT     'Astrophysics, a division of UC Berkeley's Space Science'           COMMENT     Laboratory.                                                         COMMENT     ' '                                                                 END                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             """)

5760

In [31]:
5760 / 80

72.0

In [49]:
fits.getheader("../tests/data/outputs/test_table.fits", ext=1)['NAXIS1']

923

In [11]:
my_data = open("../tests/data/outputs/test_bintable.fits", "rb").read()[get_first_header_size(open("../tests/data/outputs/test_bintable.fits", "rb").read()):]

In [15]:
fits.getheader("../tests/data/outputs/test_bintable.fits", ext=1)

XTENSION= 'BINTABLE' / Binary table extension                                   
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
PCOUNT  =                    0 / Size of special data area                      
GCOUNT  =                    1 / Only one group                                 
EXTNAME = 'ds_limits' / Name of table                                           
ORIGIN  = 'CEA/SSL UC Berkeley' / EUVE Science Archive                          
CREATOR = 'STWFITS ' / Fitsio version 11-May-1995                               
TELESCOP= 'EUVE    ' / Extreme Ultraviolet Explorer                             
INSTTYPE= 'DS/S    ' / Instrument type (DS/S, SCANNER)                          
INSTRUME= 'DS/S    ' / Deep Survey - Spectrometer                               
DETNAM  = 'DS      ' / Deep Survey                                              
OBJECT  = 'NGC 4151' / Name 

In [13]:
my_data[0:8]

b'dsq1sf  '

In [51]:
len("        1.826255233634E2 ")

25